# Computer Exercises 

In [1]:
import numpy as np
import sympy as sym
import scipy as sci
import matplotlib.pyplot as plt

from numpy import pi, cos
from sympy import symbols
from scipy.signal import welch 

%matplotlib inline
%config InlineBackend.figure_format = 'pdf'

## Power spectral density, periodogram

$$x(t) = CN(t) + \sum_{i=1}^4 A_i \cos(\omega_i t)$$

In [2]:
A = [250, 34, 8 , 10]

fn = 10
w0 = 2*pi*fn 
w1 = w0/100
w4 = w0/500
W = [w1, 4.3*w1, 4.7*w1, w4]

In [3]:
def t(NT): # Time
    t = np.linspace(0, NT/fn, NT)
    return t 

def CN(NT):
    CN = np.random.normal(0, 400**0.5, int(NT))
    return CN

def s(t): # The summation
    s = 0
    for i in range(len(A)-1):
        s += A[i] * cos(W[i]*t) 
    return s
                        
def S(NT): # Array of summation values
    S = s(t(NT))
    return S  

def x(NT): # Array of x(t) values
    x = CN(NT) + S(NT)
    return x

In [4]:
NT = [20000, 2000]

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))
ax1.plot(t(NT[0]), x(NT[0]))
ax2.plot(t(NT[1]), x(NT[1]))
ax1.set_title('$x(t) = CN(t) + \sum_{i=1}^4 A_i \cos(\omega_i t)$ for time series length (NT) of 20,000 sampled at 10Hz', fontsize=14)
ax2.set_title('$x(t) = CN(t) + \sum_{i=1}^4 A_i \cos(\omega_i t)$ for time series length (NT) of 2000 sampled at 10Hz', fontsize=14)
ax1.set_xlabel('Time ($s$)', fontsize=12)
ax2.set_xlabel('Time ($s$)', fontsize=12)

fig.tight_layout()

<Figure size 864x720 with 2 Axes>

### Step-by-step calculation of PSD:

#### For NT = 20000

In [5]:
# Fourier transform: 
FT1 = np.fft.fft(x(NT[0]))
FS1 = np.fft.fftfreq(int(NT[0]), 1/fn)

In [6]:
# Plot the Fourier Transform :
fig, (axS1, axS2) = plt.subplots(2, 1, figsize = (10, 8))
axS1.plot(FS1, FT1.real)
axS2.plot(FS1, FT1.imag)

axS1.set_title('Real part of the Fourier transform of $x(t)$ ($Re [ F \{x(t)\} ]$) for time series of length 20,000')
axS2.set_title('Imaginary part of the Fourier transform of $x(t)$ ($Im [ F \{x(t)\} ]$) for time series of length 20,000')

axS1.set_xlabel('Frequency (Hz)')
axS2.set_xlabel('Frequency (Hz)')

axS1.set_ylabel('$Re[X(f)]$')
axS2.set_ylabel('$Im[X(f)]$')

fig.tight_layout()

<Figure size 720x576 with 2 Axes>

In [7]:
# Plot the 1-sided spectral density function of the Fourier transform 
fig, (axG1, axG2) = plt.subplots(2, 1, figsize = (10, 8))
axG1.loglog(FS1[0:1000], np.abs(4*FT1[0:1000].real))
axG2.loglog(FS1[0:1000], np.abs(4*FT1[0:1000].imag))

axG1.set_title('Real part of the one-sided power spectral density function of $x(t)$ ($Re[G_{xx}(f)]$) for time series of length 20,000')
axG2.set_title('Imaginary part of the one-sided power spectral density function of $x(t)$ ($Im[G_{xx}(f)]$) for time series of length 20,000')

axG1.set_xlabel('Frequency (Hz)')
axG2.set_xlabel('Frequency (Hz)')

axG1.set_ylabel('$Re[G_{xx}(f)]$')
axG2.set_ylabel('$Im[G_{xx}(f)]$')

fig.tight_layout()

<Figure size 720x576 with 2 Axes>

#### For NT = 2000

In [8]:
# Fourier transform: 
FT2 = np.fft.fft(x(NT[1]))
FS2 = np.fft.fftfreq(int(NT[1]), 1/fn)

In [9]:
# Plot the Fourier transform:
fig, (axS3, axS4) = plt.subplots(2, 1, figsize = (10, 8))
axS3.plot(FS2, FT2.real)
axS4.plot(FS2, FT2.imag)
axS3.set_title('Real part of the Fourier transform of $x(t)$ ($Re [ F \{x(t)\} ]$) for time series of length 2000')
axS4.set_title('Imaginary part of the Fourier transform of $x(t)$ ($Im [ F \{x(t)\} ]$) for time series of length 2000')

axS3.set_xlabel('Frequency (Hz)')
axS4.set_xlabel('Frequency (Hz)')

axS3.set_ylabel('$Re[X(f)]$')
axS4.set_ylabel('$Im[X(f)]$')

fig.tight_layout()

<Figure size 720x576 with 2 Axes>

In [10]:
# Plot the 1-sided spectral density function of the Fourier transform 
fig, (axG3, axG4) = plt.subplots(2, 1, figsize = (10, 8))
axG3.loglog(FS2[0:1000], np.abs(4*FT2[0:1000].real))
axG4.loglog(FS2[0:1000], np.abs(4*FT2[0:1000].imag))
axG3.set_title('Real part of the one-sided power spectral density function of $x(t)$ ($Re[G_{xx}(f)]$) for time series of length 2000')
axG4.set_title('Imaginary part of the one-sided power spectral density function of $x(t)$ ($Im[G_{xx}(f)]$) for time series of length 2000')

axG3.set_xlabel('Frequency (Hz)')
axG4.set_xlabel('Frequency (Hz)')

axG3.set_ylabel('$Re[G_{xx}(f)]$')
axG4.set_ylabel('$Im[G_{xx}(f)]$')

fig.tight_layout()

<Figure size 720x576 with 2 Axes>

#### Questions: 

##### 1) Does increasing the record length improve the estimate? 



##### 2) How does the record length affect the frequency resolution?

### Use built in function of pwelch (well...the Python equivalent):

#### In MatLab: 

\[PP_M1, fq_M1\] = pwelch(xx1, NFFT1, NFFT1/2, NFFT1, fsamp, 'onesided');

"Returns a frequency vector in cylcles per unit time" (via MatLab documentation) 

xx1 = Time series 

NFFT = Number of DFT points 

fsamp = Sampling frequency 

'onesided' =  Returns onesided power spectral density of x(t) 


#### In Python: 

scipy.signal.welch(x, fs=1.0, window='hanning', nperseg=256, noverlap=None, nfft=None, detrend='constant', return_onesided=True, scaling='density', axis=-1)

Returns power spectral density (or power spectrum) of x(t), and array of sample fequencies

x = Time series 

fs = Sampling frequency (default = 1 Hz)

window = Tapered window (default = Hanning) 

nperseg = Length of segments (default = 256) 

noverlap = Amount of overlapping datapoints (default = None) 

nfft = Length of the fast Fourier transform 

detrend = Options to detrend each segment (default = constant) 

return_onesided = Option to return onesided function or not (default = True) 

scaling = Option to select between computing the power spectral density, or the power spectrum (default = density) 

axis = Axis used to compute the periodogram (default = -1 the last axis) 

In [40]:
ps1 = welch(x(NT[0]), 10, nperseg=5000, scaling='spectrum') 

In [41]:
plt.loglog(ps1[1])

<Figure size 432x288 with 1 Axes>